<a href="https://colab.research.google.com/github/kimhwijin/HandsOnMachineLearing/blob/main/NLP_RNN_and_Attention_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras 
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
dataset_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare", dataset_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [ ]:
print(shakespeare_text[:148])

In [ ]:
"".join(sorted((set(shakespeare_text.lower()))))

# Tokenizer

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [ ]:
print(tokenizer.word_index)
print(tokenizer.texts_to_sequences(["Shakespeare@e"]))
print(tokenizer.sequences_to_texts([[8, 7, 5, 25, 2, 8, 23, 2, 5, 9, 2]]))

In [ ]:
max_id = len(tokenizer.word_index)
print(max_id)
dataset_size = tokenizer.document_count
print(dataset_size)
print(sum(tokenizer.word_counts.values()))

# Dataset

In [ ]:
#index가 1 부터시작해서 -1 을 해줘서 0 부터로 조정함
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [ ]:
#수백만개의 1D 차원 시퀀스 데이터를 window() 메서드로 작은 많은 텍스트로 변환한다.
n_steps = 100
window_length = n_steps + 1 # target = 다음 1 글자 input
#shift = 1 이면, 데이터셋을 꽉차게 사용한다. 0~100 , 1~101 , ...
#window는 데이터셋을 만들어서, 리스트의 리스트 같이, 중첩 데이터셋을 만듬
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [ ]:
for a in dataset.take(1):
    #데이터셋
    print(len(a), a)

In [ ]:
#중첩 데이터셋을 덴서를 포함한 데이터셋으로 변경함.
dataset = dataset.flat_map(lambda window : window.batch(window_length))
for a in dataset.take(1):
    print(len(a), a)

In [ ]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
#이전 100개의 글자와 타깃값 1글자를 분리함
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [ ]:
#각 글자를 one hot 벡터로 치환한다.
#원래는 글자수가 많으면 임베딩을 사용함
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
for x, y in dataset.take(1):
    print(x[0])
    print(y[0])


# Model

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)

Epoch 1/10
31368/31368 [==============================] - 489s 15ms/step - loss: 1.6206
Epoch 2/10
31368/31368 [==============================] - 468s 15ms/step - loss: 1.5369
Epoch 3/10
31368/31368 [==============================] - 484s 15ms/step - loss: 1.5171
Epoch 4/10
31368/31368 [==============================] - 492s 16ms/step - loss: 1.5054
Epoch 5/10
31368/31368 [==============================] - 490s 16ms/step - loss: 1.4978
Epoch 6/10
31368/31368 [==============================] - 489s 16ms/step - loss: 1.4927
Epoch 7/10
31368/31368 [==============================] - 483s 15ms/step - loss: 1.4895
Epoch 8/10
31368/31368 [==============================] - 462s 15ms/step - loss: 1.4868
Epoch 9/10
 6776/31368 [=====>........................] - ETA: 6:01 - loss: 1.4824